# Marching Tetrahedra
https://michelanders.blogspot.com/2012/02/marching-tetrahedrons-in-python.html

### Classes:

In [1]:
class Vector:  # struct XYZ
	def __init__(self, x, y, z):
		self.x = x
		self.y = y
		self.z = z
	
	def __str__(self):
		return str(self.x) + " " + str(self.y) + " " + str(self.z)


class Gridcell:  # struct GRIDCELL
	def __init__(self, p, n, val):
		self.p = p  # p=[8]
		self.n = n  # n=[8]
		self.val = val  # val=[8]


class Triangle:  # struct TRIANGLE
	def __init__(self, p1, p2, p3):
		self.p = [p1, p2, p3]  # vertices
	
	# return triangle as an ascii STL facet
	def __str__(self):
		return """
facet normal 0 0 0
outer loop 
    vertex %s 
    vertex %s 
    vertex %s 
endloop 
endfacet
""" % (self.p[0], self.p[1], self.p[2])

# return a 3d list of values



### Cases
Different cases that can occure with the tetrahedra's.

In [2]:

def t000F(g, iso, v0, v1, v2, v3):
	return []


def t0E01(g, iso, v0, v1, v2, v3):
	return [Triangle(
		VertexInterp(iso, g.p[v0], g.p[v1], g.val[v0], g.val[v1]),
		VertexInterp(iso, g.p[v0], g.p[v2], g.val[v0], g.val[v2]),
		VertexInterp(iso, g.p[v0], g.p[v3], g.val[v0], g.val[v3]))
	]


def t0D02(g, iso, v0, v1, v2, v3):
	return [Triangle(
		VertexInterp(iso, g.p[v1], g.p[v0], g.val[v1], g.val[v0]),
		VertexInterp(iso, g.p[v1], g.p[v3], g.val[v1], g.val[v3]),
		VertexInterp(iso, g.p[v1], g.p[v2], g.val[v1], g.val[v2]))
	]


def t0C03(g, iso, v0, v1, v2, v3):
	tri = Triangle(
		VertexInterp(iso, g.p[v0], g.p[v3], g.val[v0], g.val[v3]),
		VertexInterp(iso, g.p[v0], g.p[v2], g.val[v0], g.val[v2]),
		VertexInterp(iso, g.p[v1], g.p[v3], g.val[v1], g.val[v3]))
	return [tri, Triangle(
		tri.p[2],
		VertexInterp(iso, g.p[v1], g.p[v2], g.val[v1], g.val[v2]),
		tri.p[1])
	        ]


def t0B04(g, iso, v0, v1, v2, v3):
	return [Triangle(
		VertexInterp(iso, g.p[v2], g.p[v0], g.val[v2], g.val[v0]),
		VertexInterp(iso, g.p[v2], g.p[v1], g.val[v2], g.val[v1]),
		VertexInterp(iso, g.p[v2], g.p[v3], g.val[v2], g.val[v3]))
	]


def t0A05(g, iso, v0, v1, v2, v3):
	tri = Triangle(
		VertexInterp(iso, g.p[v0], g.p[v1], g.val[v0], g.val[v1]),
		VertexInterp(iso, g.p[v2], g.p[v3], g.val[v2], g.val[v3]),
		VertexInterp(iso, g.p[v0], g.p[v3], g.val[v0], g.val[v3]))
	return [tri, Triangle(
		tri.p[0],
		VertexInterp(iso, g.p[v1], g.p[v2], g.val[v1], g.val[v2]),
		tri.p[1])
	        ]


def t0906(g, iso, v0, v1, v2, v3):
	tri = Triangle(
		VertexInterp(iso, g.p[v0], g.p[v1], g.val[v0], g.val[v1]),
		VertexInterp(iso, g.p[v1], g.p[v3], g.val[v1], g.val[v3]),
		VertexInterp(iso, g.p[v2], g.p[v3], g.val[v2], g.val[v3]))
	return [tri,
	        Triangle(
		        tri.p[0],
		        VertexInterp(iso, g.p[v0], g.p[v2], g.val[v0], g.val[v2]),
		        tri.p[2])
	        ]


def t0708(g, iso, v0, v1, v2, v3):
	return [Triangle(
		VertexInterp(iso, g.p[v3], g.p[v0], g.val[v3], g.val[v0]),
		VertexInterp(iso, g.p[v3], g.p[v2], g.val[v3], g.val[v2]),
		VertexInterp(iso, g.p[v3], g.p[v1], g.val[v3], g.val[v1]))
	]

trianglefs = {7: t0708, 8: t0708, 9: t0906, 6: t0906, 10: t0A05, 5: t0A05, 11: t0B04, 4: t0B04, 12: t0C03, 3: t0C03, 13: t0D02, 2: t0D02, 14: t0E01, 1: t0E01, 0: t000F, 15: t000F}


def PolygoniseTri(g, iso, v0, v1, v2, v3):
	triangles = []
	
	#   Determine which of the 16 cases we have given which vertices
	#   are above or below the isosurface
	
	triindex = 0;
	if g.val[v0] < iso: triindex |= 1
	if g.val[v1] < iso: triindex |= 2
	if g.val[v2] < iso: triindex |= 4
	if g.val[v3] < iso: triindex |= 8
	
	return trianglefs[triindex](g, iso, v0, v1, v2, v3)


def VertexInterp(isolevel, p1, p2, valp1, valp2):
	if abs(isolevel - valp1) < 0.00001:
		return (p1);
	if abs(isolevel - valp2) < 0.00001:
		return (p2);
	if abs(valp1 - valp2) < 0.00001:
		return (p1);
	mu = (isolevel - valp1) / (valp2 - valp1)
	return Vector(p1.x + mu * (p2.x - p1.x), p1.y + mu * (p2.y - p1.y), p1.z + mu * (p2.z - p1.z))


### Support Functions

In [3]:

def readdata(f=lambda x, y, z: x * x + y * y + z * z, size=5.0, steps=11):
    """ 
    Size is how big volume is?
    Steps is multiplier 
    """
	m = int(steps / 2)
	ki = []
	for i in range(steps):
		kj = []
		for j in range(steps):
			kd = []
			for k in range(steps):
				kd.append(f(size * (i - m) / m, size * (j - m) / m, size * (k - m) / m))
			kj.append(kd)
		ki.append(kj)
	return ki

from math import cos, exp, atan2

def export_triangles(triangles,output_name="output"):  # stl format
	with open(output_name+".stl", "w+") as f:
		f.write("solid points\n")
		for tri in triangles:
			f.write(tri.__str__())
		f.write("endsolid points")
    


### Load the data

In [9]:
# Load data
import nibabel
from loadnii import segment_labels

volume_file_path = "data/ctscan1/volume-3.nii.gz"
label_file_path = "data/ctscan1/labels-3.nii.gz"

labels = {"liver":1, "bladder":2, "lungs":3, "kidneys":4,"bones":5, "brain":6}

target_label = labels["bones"]
minlabel = target_label - .1
maxlabel = target_label + .1

new_image = segment_labels(volume_file_path, label_file_path, minlabel, maxlabel)
segmented_label_data = new_image.get_fdata()
print(f"Loaded volume of shape: {segmented_label_data.shape} Seperation value was: {minlabel} - {maxlabel}")


The label and image shapes are the same its (512, 512, 123)

The volume is in label4


### Iso Functions
These are functions that are called on each voxel. The return value is matched with the isovalue. 

In [23]:

def lobes(x, y, z):
    try:
        theta = atan2(x, y)  # sin t = o
    except:
        theta = 0
    try:
        phi = atan2(z, y)
    except:
        phi = 0
    r = x * x + y * y + z * z
    ct = cos(theta)
    cp = cos(phi)
    res = ct * ct * cp * cp * exp(-r / 10)
    return res

def f(x,y,z):
    x, y, z = int(x), int(y), int(z)
    print(x,y,z)
    try:
        return label4[x][y][z]
    except Exception as e:
        print(e)
        return 0

**Main code:**

In [34]:
# data = readdata(lobes,1,11)
# isolevel = 0.1

# data = readdata(f,100,-50)
isolevel = 3
data = segmented_label_data

# print(data)

triangles = []
for i in range(len(data) - 1):
    print(f"i:{i} t:{len(triangles)}",end=", ")
    for j in range(len(data[i]) - 1):
        for k in range(len(data[i][j]) - 1):
            p = [None] * 8
            val = [None] * 8

            p[0] = Vector(i, j, k)
            val[0] = data[i][j][k]
            p[1] = Vector(i + 1, j, k)
            val[1] = data[i + 1][j][k]
            p[2] = Vector(i + 1, j + 1, k)
            val[2] = data[i + 1][j + 1][k]
            p[3] = Vector(i, j + 1, k)
            val[3] = data[i][j + 1][k]
            p[4] = Vector(i, j, k + 1)
            val[4] = data[i][j][k + 1]
            p[5] = Vector(i + 1, j, k + 1)
            val[5] = data[i + 1][j][k + 1]
            p[6] = Vector(i + 1, j + 1, k + 1)
            val[6] = data[i + 1][j + 1][k + 1]
            p[7] = Vector(i, j + 1, k + 1)
            val[7] = data[i][j + 1][k + 1]
            grid = Gridcell(p, [], val)
            triangles.extend(PolygoniseTri(grid, isolevel, 0, 2, 3, 7))
            triangles.extend(PolygoniseTri(grid, isolevel, 0, 2, 6, 7))
            triangles.extend(PolygoniseTri(grid, isolevel, 0, 4, 6, 7))
            triangles.extend(PolygoniseTri(grid, isolevel, 0, 6, 1, 2))
            triangles.extend(PolygoniseTri(grid, isolevel, 0, 6, 1, 4))
            triangles.extend(PolygoniseTri(grid, isolevel, 5, 6, 1, 4))

print(f"Found {len(triangles)} triangles")
output = "nicebones"
export_triangles(triangles, output)
print(f"Exported {output}")

i:0 t:0, i:1 t:0, i:2 t:0, i:3 t:0, i:4 t:0, i:5 t:0, i:6 t:36, i:7 t:202, i:8 t:424, i:9 t:702, i:10 t:1094, i:11 t:1576, i:12 t:2196, i:13 t:2936, i:14 t:3756, i:15 t:4808, i:16 t:6046, i:17 t:7408, i:18 t:8872, i:19 t:10476, i:20 t:12130, i:21 t:13788, i:22 t:15506, i:23 t:17216, i:24 t:19018, i:25 t:20768, i:26 t:22716, i:27 t:25050, i:28 t:27716, i:29 t:30570, i:30 t:33578, i:31 t:36746, i:32 t:39964, i:33 t:43376, i:34 t:46788, i:35 t:50296, i:36 t:54028, i:37 t:57906, i:38 t:61908, i:39 t:65926, i:40 t:69960, i:41 t:74094, i:42 t:78360, i:43 t:82698, i:44 t:87416, i:45 t:92528, i:46 t:97872, i:47 t:103214, i:48 t:108730, i:49 t:114488, i:50 t:120962, i:51 t:127692, i:52 t:134490, i:53 t:141476, i:54 t:148404, i:55 t:155654, i:56 t:163356, i:57 t:171356, i:58 t:179660, i:59 t:188354, i:60 t:197166, i:61 t:205924, i:62 t:214738, i:63 t:223396, i:64 t:232330, i:65 t:241576, i:66 t:251064, i:67 t:260454, i:68 t:269720, i:69 t:279176, i:70 t:289122, i:71 t:299542, i:72 t:310168, i:73

i:503 t:4284750, i:504 t:4285076, i:505 t:4285342, i:506 t:4285426, i:507 t:4285506, i:508 t:4285578, i:509 t:4285634, i:510 t:4285646, Found 4285646 triangles
Exported 2balls


### Optimise mesh

In [7]:
import pymesh
from cleanmesh import cleanup_mesh
mesh = pymesh.meshio.load_mesh("big2balls.stl")

In [6]:
meshclean= cleanup_mesh(mesh)

Removing duplicated triangles... removed 4088640
Removing obtuse triangles... removed 324334
Removing degenerative triangles... removed 4737249
Removing unused vertices... removed 0 lines
Removing duplicated vertices... removed 0


In [17]:
pymesh.meshio.save_mesh("coolbonesoptimised.obj", mesho6)